In [ ]:
# These answers are the ones found in the text.  I am comfortable enough to be able to follow and understand the answers found in the text, but not comfortable enough to create my own versions of them.  Any thing I attempted failed.

# 1. Try a Support Vector Machine regressor with various hyperparameters such as kernel="linear" or kernel="rbf".  Don't worry about what these hyperparameters mean for now.  How does the best SVR predictor perform?


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
    {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0], 'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]

svm_reg = SVR()
grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(housing_prepared, housing_labels)


In [ ]:
# Linear works best with these values in the SVR, this is due to the current c values.  Replacing the c values may end up with a different outcome.  

In [ ]:
# 2. Try replacing GridSearchCV with RandomizedSearchCV.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

param_distribs = {
    'kernel': ['linear', 'rbf'],
    'C': reciprocal (20,200000),
    'gamma': expon(scale=1.0),
}

svm_reg = SVR()
rnd_search = RandomizedSearchCV(svm_reg, param_distributions=param_distribs, n_iter=50, cv=5, scoring='neg_mean_squared_error', verbose+2, random_state=42)
rnd_search.fit(housing_prepared, housing_labels)


In [ ]:
#  This random search provided better parameters for a RBF kernel.  Random search seems to be more efficient than grid, as it finds better parameters in the same amount of time.

In [ ]:
# 3. Try adding a transformer in the preparation pipeline to select only the most important attributes.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

def indices_of_top_k(arr, k):
    return np.sort(np.argpartition (np.array(arr), -k) [-k:])

class TopFeatureSelector(BaseEstimator, TransormerMixin):
    def _init_(self, feature_importances, k):
        self.feature_importances = feature_importances
        return self.k = k
    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self
    def transform(self, X):
        return X[:, self.feature_indices_]
    
k=5
top_k_feature_indices = indices_of_top_k(feature_importances, k)
top_k_feature_indices
np.array(attributes)[top_k_feature_indices]
sorted(zip(feature_importances, attributes), reverse=True)[:k]

preparation_and_feature_selection_pipeline = Pipeline([('preparation', full_pipeline),('feature_selection', TopFeatureSelector(feature_importances, k))])
housing_prepared_top_k_features = preparation_and_feature_selection_pipeline.fit_transform(housing)

In [ ]:
# This is pretty confusing at first, but it uses the existing pipeline then runs that pipeline into a new one that adds the transformer which only grabs the valuable attributes.

In [ ]:
# Try creating a single pipeline that does the full data preparation plus the final prediction.

In [ ]:
prepare_select_and_predict_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, k)),
    ('svm_reg', SVR(**rnd_search.best_params_))
])

prepare_select_and_predict_pipeline.fit(housing, housing_labels)


In [ ]:
# The pipeline gathers all the data, and makes a prediction.

In [ ]:
# 5. Automatically explore some preparation options using GridSearchCV.

In [ ]:
param_grid = [{
    'preparation_num_imputer_strategy': ['mean', 'median', 'most_frequent'],
    'feature_selection_k': list(ranfe(1, len(feature_importances) +1))
}]

grid_search_prep = GridSearchCV(prepare_select_and_predict_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)

grid_search_prep.fit(housing, housing_labels)

In [ ]:
# The results from this indicate that most_frequent is the best preparation option.